In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import MultinomialNB
import re

In [2]:
users = pd.read_csv('data/full/train.csv')

In [6]:
users = users[['user_id','target']]

In [12]:
tweets = pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']]

In [14]:
tweets = pd.concat([pd.read_csv('data/propaganda/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/porn/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/spam/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/genuine/tweets.csv', sep='\t')[['user_id','full_text']]])

/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  int

In [17]:
tweets = tweets.drop_duplicates()

In [20]:
tweets = pd.merge(tweets, users, on='user_id')

In [26]:
tweets.drop(['user_id'], axis=1, inplace=True)

In [28]:
tweets.head()

,full_text,target
0,RT @funder: Retweetfest: Tweet this link out i...,1
1,RT @RWPUSA: So much for party labels. With Dem...,1
2,RT @funder: BREAKING: Trump just said he’s ref...,1
3,RT @robreiner: When an American President atta...,1
4,RT @SteveSchmidtSES: TRUMP disgraced the Presi...,1


In [32]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: x.replace('RT', ''))

In [39]:
def remove_rt(x):
    
    try:
        return x[x.rindex(':')+1:]
    except:
        return x

In [40]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: remove_rt(x))

In [51]:
tweets['full_text'] = tweets['full_text'].astype(str)

In [58]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(tweets.full_text).toarray()
labels = tweets.category_id
features.shape

MemoryError: 

In [42]:
X_train, X_test, y_train, y_test = train_test_split(tweets['full_text'], tweets['target'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [54]:
y_pred = clf.predict(X_test[:-1])

ValueError: could not convert string to float: ' Dear @newsbyhughes @CNN, Congratulations! Once again you are wrong. There is no “some kind of group signing” taking place. It…'

In [45]:
X_test

779853     "Some people put walls up, not to keep people ...
630020           Where were u wen I just finessin midget????
124189      @Imaginer77 @Sambyhere @Determined1776 @Grann...
706427     i think i just entered a parallel universe...a...
884188                                     //t.co/bPtZcCQJMj
157349      Bill Clinton raped a woman, then paid her $80...
636929     @ti_pac7 "All our dreams can come true, if we ...
1074869                                    //t.co/2s9VK5drMn
158578      @FoxNews Shows the extremes the left will go ...
184946      Gavin Newsom admits he had an affair with his...
167284      Yes, it is who you are. This is your party. Y...
157591      The same folks ALARMED that Trump said Russia...
653861     On nyquil? Niz. I take it as directed lol I gi...
257132      Information is being illegally given to the f...
823954     Trust people and they will be true to you; Tre...
806689     My moms said you cant run game on someone who ...
1151424                 

In [55]:
type(tweets.iloc[1191716]['full_text'])

str

SyntaxError: invalid character in identifier (<ipython-input-49-eb78a41c5af4>, line 1)